# Experimento 1: Palavras e temas dominantes


In [ ]:
# 1. Instalação de pacotes que podem não vir por padrão (normalmente o Colab já tem, mas garante)
!pip install wordcloud nltk pandas matplotlib scikit-learn

# 2. Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive

# 3. Configurações de Visualização
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

# 4. Montar o Google Drive
drive.mount('/content/drive')

# 5. Carregar os Dados
path_folder = '/content/drive/MyDrive/IFES/Disciplinas/6-periodo/Topicos-Especiais-de-Banco-de-Dados/Jogo-complete-a-letra/albuns_engenheiros_csv/musicas-analise'
all_files = glob.glob(os.path.join(path_folder, "*.csv"))

df_list = []

print(f"Encontrados {len(all_files)} arquivos de álbuns.")

for filename in all_files:
    try:
        # Tenta ler com utf-8, se falhar tenta latin-1 (comum em arquivos excel/csv antigos)
        try:
            df_temp = pd.read_csv(filename, encoding='utf-8')
        except UnicodeDecodeError:
            df_temp = pd.read_csv(filename, encoding='latin-1')

        # Padronizar nomes das colunas para minúsculas para evitar erros (Titulo vs titulo)
        df_temp.columns = df_temp.columns.str.lower()

        # Verifica se as colunas necessárias existem
        if 'titulo' in df_temp.columns and 'letra' in df_temp.columns:
            df_list.append(df_temp[['titulo', 'letra']])
        else:
            print(f"Aviso: Arquivo {os.path.basename(filename)} ignorado (colunas incorretas).")

    except Exception as e:
        print(f"Erro ao ler {filename}: {e}")

# Concatenar tudo em um único DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Total de músicas carregadas: {len(df)}")
df.head()

In [ ]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string

# Configurações iniciais
nltk.download('stopwords')
nltk.download('punkt')

# --- 1. Carregamento e Pré-processamento [cite: 15] ---
# SUBSTITUA 'seu_arquivo.csv' PELO CAMINHO DO SEU DATASET REAL
# O dataset deve ter uma coluna com o texto das letras, aqui chamada de 'letra'
try:
    df = pd.read_csv('seu_arquivo.csv')
except:
    # Dados fictícios para exemplo caso não tenha o arquivo agora
    data = {'titulo': ['Infinita Highway', 'Refrão de Bolero'],
            'letra': ['Você me faz correr demais nesta estrada', 'Um dia me disseram que as nuvens não eram de algodão']}
    df = pd.DataFrame(data)

def preprocessar(texto):
    # Converter para minúsculas [cite: 17]
    texto = texto.lower()
    # Remover pontuação [cite: 17]
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Tokenização simples (split)
    tokens = texto.split()
    # Remover Stopwords [cite: 18]
    stops = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stops]
    return tokens

# Aplicar pré-processamento
df['tokens'] = df['letra'].apply(preprocessar)
df['texto_limpo'] = df['tokens'].apply(lambda x: ' '.join(x))

# --- 2. Análise 1: Contagem Simples e WordCloud [cite: 19] ---
todos_tokens = [word for lista in df['tokens'] for word in lista]
contagem = Counter(todos_tokens)

# Top-20 Termos [cite: 22]
print("--- TOP 20 PALAVRAS MAIS COMUNS ---")
print(contagem.most_common(20))

# Nuvem de Palavras [cite: 21]
wc = WordCloud(width=800, height=400, background_color='white', max_words=100).generate_from_frequencies(contagem)
plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("Nuvem de Palavras - Engenheiros do Hawaii")
plt.show()

# --- 3. Análise 2: TF-IDF [cite: 23] ---
# Identificar palavras importantes para músicas específicas [cite: 25]
tfidf = TfidfVectorizer(stop_words=list(stopwords.words('portuguese')))
matrix_tfidf = tfidf.fit_transform(df['texto_limpo'])
feature_names = tfidf.get_feature_names_out()

# Média dos scores TF-IDF para achar termos relevantes globalmente [cite: 27]
mean_tfidf = matrix_tfidf.mean(axis=0)
tfidf_scores = [(feature_names[i], mean_tfidf[0, i]) for i in range(mean_tfidf.shape[1])]
tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

print("\n--- TOP 20 TERMOS (SCORE TF-IDF) ---")
for termo, score in tfidf_scores[:20]:
    print(f"{termo}: {score:.4f}")